# Building a RAG Agent with LangGraph: Complete Tutorial


For more detail Langgraph Agent learning, you can check this repo: https://github.com/ScottLL/langgraph_lib

## 1. Introduction to RAG Agents

### What is a RAG Agent?

A **RAG (Retrieval-Augmented Generation) Agent** combines the power of Large Language Models (LLMs) with external knowledge retrieval capabilities. Unlike standalone LLMs that are limited to their training data, RAG agents can:

- **Access up-to-date information** from external documents
- **Perform multi-step reasoning** with retrieved context
- **Maintain conversation state** across interactions
- **Use specialized tools** for document retrieval

### Key Components We'll Build:

1. **Document Processor**: Loads and chunks PDF documents
2. **Vector Store**: Creates searchable embeddings of document chunks
3. **Retrieval Tool**: Searches for relevant information
4. **LLM Agent**: Reasons with retrieved information
5. **Graph Orchestrator**: Manages the agent workflow

### Architecture Overview:

```
User Query → LLM Agent → Retrieval Tool → Vector Store → PDF Documents
     ↑                        ↓
     └── Final Response ← LLM Agent ← Retrieved Context
```

## 2. Environment Setup

#### 2.1 Start Anaconda to enter the working directory:
cd C:\Users\ch939\Downloads\LLMBootCampCodes\Week10

#### 2.2 Open VS Code terminal
code .

#### 2.3 Create virtual environment
conda create -n langchain_venv python=3.10.18

#### 2.4 Activate the virtual environment
conda activate langchain_venv

#### 2.5 Upgrade pip
pip install --upgrade pip

#### 2.6 Install PyTorch (GPU build for CUDA 12.8)
Since my RTX 4070 SUPER + CUDA 12.8 is very recent, install via PyTorch’s official channel:

However, pytorch-cuda=12.8 is not a valid package name or version in the PyTorch or NVIDIA conda channels.

conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia

This will install:
* PyTorch (GPU-enabled)
* CUDA 12.8 runtime (no need to match driver manually — it uses your system driver)
* cuDNN (already included in package)

#### 2.7 Install Jupyter + extras
conda install jupyter 


conda install -c conda-forge numpy scipy pandas  

pip install langchain langchain-openai langchain-community langgraph chromadb pypdf python-dotenv langchain_chroma

pip install ipywidgets

In [7]:
# Import all necessary libraries
from dotenv import load_dotenv
import os
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, ToolMessage
from operator import add as add_messages
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.tools import tool

# Load environment variables (make sure you have OPENAI_API_KEY in your .env file)
load_dotenv()

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


**💡 What's happening here?**
- **LangChain**: Framework for building LLM applications
- **LangGraph**: Creates state-based agent workflows
- **ChromaDB**: Vector database for storing embeddings
- **OpenAI**: LLM and embedding models

## 3. Model Initialization 

Let's set up our LLM and embedding models:

In [8]:
# Initialize the Language Model
llm = ChatOpenAI(
    model="gpt-4o", 
    temperature=0  # Low temperature for more deterministic responses
)

# Initialize the Embedding Model
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",  # Efficient embedding model
)

print("🤖 LLM initialized: GPT-4o")
print("🔍 Embeddings initialized: text-embedding-3-small")

🤖 LLM initialized: GPT-4o
🔍 Embeddings initialized: text-embedding-3-small


**🔍 Key Parameters:**
- **Temperature = 0**: Makes responses more deterministic and factual
- **text-embedding-3-small**: Cost-effective embedding model for document similarity

## 4. Document Loading and Processing

Now let's load and process our PDF document:

In [9]:
# Define the path to your PDF document
pdf_path = "Stock_Market_Performance_2024.pdf"

# Safety check: Verify PDF exists
if not os.path.exists(pdf_path):
    raise FileNotFoundError(f"❌ PDF file not found: {pdf_path}")
    
print(f"📄 Found PDF: {pdf_path}")

📄 Found PDF: Stock_Market_Performance_2024.pdf


In [10]:
# Load the PDF document
pdf_loader = PyPDFLoader(pdf_path)

try:
    pages = pdf_loader.load()
    print(f"✅ PDF loaded successfully!")
    print(f"📊 Document contains {len(pages)} pages")
    
    # Preview first page content (first 200 characters)
    if pages:
        print(f"📝 First page preview: {pages[0].page_content[:200]}...")
        
except Exception as e:
    print(f"❌ Error loading PDF: {e}")
    raise

✅ PDF loaded successfully!
📊 Document contains 9 pages
📝 First page preview: Stock Market Performance in 2024
U.S. Market Overview
The year 2024 was a remarkably strong one for equities, with the U.S. stock market extending the
robust gains seen in the prior year. The benchmar...


**🔄 What's happening?**
- **PyPDFLoader**: Extracts text from each page of the PDF
- **Error handling**: Ensures robust document loading
- **Content preview**: Shows what was extracted

## 5. Text Chunking 

Large documents need to be split into smaller, manageable chunks:

In [12]:
# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # Maximum characters per chunk
    chunk_overlap=200     # Overlap between chunks to maintain context
)

# Split the document into chunks
pages_split = text_splitter.split_documents(pages)

print(f"📄 Original pages: {len(pages)}")
print(f"🔍 Total chunks created: {len(pages_split)}")
print(f"📏 Average chunk size: {sum(len(chunk.page_content) for chunk in pages_split) // len(pages_split)} characters")

# Preview a sample chunk
if pages_split:
    print(f"\n📝 Sample chunk:\n{pages_split[0].page_content[:300]}...")

📄 Original pages: 9
🔍 Total chunks created: 24
📏 Average chunk size: 868 characters

📝 Sample chunk:
Stock Market Performance in 2024
U.S. Market Overview
The year 2024 was a remarkably strong one for equities, with the U.S. stock market extending the
robust gains seen in the prior year. The benchmark S&P 500 index delivered roughly a 25% total
return for 2024 (around +23% in price terms)
. This ma...


**🎯 Chunking Strategy:**
- **Chunk size (1000)**: Balance between context and retrieval precision
- **Overlap (200)**: Prevents information loss at chunk boundaries
- **Recursive splitting**: Maintains semantic coherence

## 6. Vector Store Creation

Create a ChromaDB vector store to enable semantic search:

In [13]:
# Set up ChromaDB configuration
persist_directory = "./chroma_db"  # Local directory for vector store
collection_name = "stock_market"   # Collection name for our documents

# Create directory if it doesn't exist
if not os.path.exists(persist_directory):
    os.makedirs(persist_directory)
    print(f"📁 Created directory: {persist_directory}")

try:
    # Create the ChromaDB vector store
    vectorstore = Chroma.from_documents(
        documents=pages_split,
        embedding=embeddings,
        persist_directory=persist_directory,
        collection_name=collection_name
    )
    print(f"✅ ChromaDB vector store created successfully!")
    print(f"📊 Indexed {len(pages_split)} document chunks")
    
except Exception as e:
    print(f"❌ Error setting up ChromaDB: {str(e)}")
    raise

📁 Created directory: ./chroma_db
✅ ChromaDB vector store created successfully!
📊 Indexed 24 document chunks


**💾 Vector Store Benefits:**
- **Semantic search**: Find similar content, not just keyword matches
- **Persistent storage**: Database saves to disk for reuse
- **Efficient retrieval**: Fast similarity search across large documents

## 7. Building the Retrieval Tool

Create a retriever and wrap it in a tool that the agent can use:

In [14]:
# Create the retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}  # Return top 5 most similar chunks
)

print(f"🔍 Retriever configured to return top 5 similar chunks")

🔍 Retriever configured to return top 5 similar chunks


In [15]:
# Define the retrieval tool using LangChain's @tool decorator
@tool
def retriever_tool(query: str) -> str:
    """
    This tool searches and returns information from the Stock Market Performance 2024 document.
    Use this tool when you need to find specific information about stock market data, trends, or analysis.
    """
    
    # Perform the similarity search
    docs = retriever.invoke(query)
    
    # Handle case where no documents are found
    if not docs:
        return "I found no relevant information in the Stock Market Performance 2024 document."
    
    # Format the retrieved documents
    results = []
    for i, doc in enumerate(docs):
        results.append(f"Document {i+1}:\n{doc.page_content}")
    
    return "\n\n".join(results)

# Test the retrieval tool
print("🛠️ Retrieval tool created successfully!")
print("📋 Tool description:", retriever_tool.description)

🛠️ Retrieval tool created successfully!
📋 Tool description: This tool searches and returns information from the Stock Market Performance 2024 document.
Use this tool when you need to find specific information about stock market data, trends, or analysis.


**🔧 Tool Features:**
- **Similarity search**: Finds semantically relevant content
- **Formatted output**: Returns structured, numbered results
- **Error handling**: Graceful handling of empty results

## 8. Agent State and Architecture

Define the agent's state and workflow structure:

In [16]:
# Define the agent's state schema
class AgentState(TypedDict):
    """
    The state of our agent conversation.
    Messages are accumulated using the add_messages function.
    """
    messages: Annotated[Sequence[BaseMessage], add_messages]

print("📋 Agent state schema defined")

📋 Agent state schema defined


In [17]:
# Create tools list and bind to LLM
tools = [retriever_tool]
llm_with_tools = llm.bind_tools(tools)

# Create tools dictionary for easy lookup
tools_dict = {tool.name: tool for tool in tools}

print(f"🔧 Available tools: {list(tools_dict.keys())}")
print(f"🤖 LLM bound with {len(tools)} tools")

🔧 Available tools: ['retriever_tool']
🤖 LLM bound with 1 tools


**🏗️ Architecture Components:**
- **AgentState**: Maintains conversation history
- **Tool binding**: Allows LLM to call our retrieval function
- **Tools dictionary**: Enables dynamic tool execution

## 9. Agent Functions 

Define the core agent functions for reasoning and tool execution:

In [22]:
def should_continue(state: AgentState) -> bool:
    """
    Determines if the agent should continue with tool calls.
    Returns True if the last message contains tool calls, False otherwise.
    """
    last_message = state['messages'][-1]
    has_tool_calls = hasattr(last_message, 'tool_calls') and len(last_message.tool_calls) > 0
    
    print(f"🤔 Should continue? {has_tool_calls}")
    return has_tool_calls

print("✅ Continuation logic defined")

✅ Continuation logic defined


In [21]:
# System prompt for the agent
system_prompt = """
You are an intelligent AI assistant specialized in analyzing Stock Market Performance data from 2024.

Your capabilities:
- Use the retriever tool to search through the Stock Market Performance 2024 document
- Provide accurate, data-driven answers based on the retrieved information
- Make multiple tool calls if needed to gather comprehensive information
- Always cite specific parts of the documents you reference

Instructions:
- When answering questions, first search for relevant information using the retriever tool
- If you need additional context, make follow-up searches with different keywords
- Always provide specific citations from the documents
- Be clear about the source of your information
"""

print("📜 System prompt configured")

📜 System prompt configured


In [20]:
def call_llm(state: AgentState) -> AgentState:
    """
    The main reasoning function that calls the LLM with current state.
    """
    messages = list(state['messages'])
    # Add system prompt to the beginning
    messages = [SystemMessage(content=system_prompt)] + messages
    
    print("🧠 Calling LLM for reasoning...")
    response = llm_with_tools.invoke(messages)
    
    print(f"💬 LLM response type: {type(response).__name__}")
    if hasattr(response, 'tool_calls') and response.tool_calls:
        print(f"🔧 LLM wants to use {len(response.tool_calls)} tools")
    
    return {'messages': [response]}

print("✅ LLM function defined")

✅ LLM function defined


In [18]:
def take_action(state: AgentState) -> AgentState:
    """
    Executes tool calls from the LLM's response.
    """
    last_message = state['messages'][-1]
    tool_calls = last_message.tool_calls
    
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call['name']
        tool_args = tool_call['args']
        
        print(f"🔧 Executing tool: {tool_name}")
        print(f"📝 Query: {tool_args.get('query', 'No query provided')}")
        
        if tool_name not in tools_dict:
            print(f"❌ Tool '{tool_name}' not found!")
            result = f"Error: Tool '{tool_name}' does not exist."
        else:
            try:
                result = tools_dict[tool_name].invoke(tool_args.get('query', ''))
                print(f"✅ Tool executed successfully. Result length: {len(str(result))} characters")
            except Exception as e:
                print(f"❌ Tool execution error: {e}")
                result = f"Error executing tool: {e}"
        
        # Create tool message
        tool_message = ToolMessage(
            tool_call_id=tool_call['id'], 
            name=tool_name, 
            content=str(result)
        )
        results.append(tool_message)

    print(f"🔄 Returning {len(results)} tool results to LLM")
    return {'messages': results}

print("✅ Tool execution function defined")

✅ Tool execution function defined


**🔄 Agent Workflow:**
1. **call_llm**: LLM reasons and decides on actions
2. **should_continue**: Checks if tools need to be called  
3. **take_action**: Executes the chosen tools
4. **Loop back**: Returns to LLM with tool results

## 10. Graph Construction

Build the LangGraph workflow:

In [23]:
# Create the state graph
graph = StateGraph(AgentState)

# Add nodes to the graph
graph.add_node("llm", call_llm)
graph.add_node("retriever_agent", take_action)

print("📊 Graph nodes added: llm, retriever_agent")

📊 Graph nodes added: llm, retriever_agent


In [25]:
# Add conditional edges
graph.add_conditional_edges(
    "llm",                    # Start from LLM
    should_continue,          # Decision function
    {True: "retriever_agent", False: END}  # If True -> tools, if False -> end
)

# Add edge from tools back to LLM
graph.add_edge("retriever_agent", "llm")

# Set the entry point
graph.set_entry_point("llm")

print("🔗 Graph edges and entry point configured")

🔗 Graph edges and entry point configured


In [26]:
# Compile the graph
rag_agent = graph.compile()

print("✅ RAG Agent compiled successfully!")
print("🎯 Agent is ready to handle queries!")

✅ RAG Agent compiled successfully!
🎯 Agent is ready to handle queries!


**🎯 Graph Flow:**
```
Start → LLM → Decision → [Tools] → LLM → End
                ↓         ↑
                End   Results
```

## 11. Testing the Agent

Let's test our RAG agent with a sample query:

In [27]:
def test_agent_query(question: str):
    """
    Test function to demonstrate agent capabilities
    """
    print(f"\n🔍 Testing Query: '{question}'")
    print("=" * 50)
    
    # Create human message
    messages = [HumanMessage(content=question)]
    
    # Run the agent
    result = rag_agent.invoke({"messages": messages})
    
    # Print the final response
    final_response = result['messages'][-1].content
    print(f"\n🤖 Agent Response:\n{final_response}")
    print("=" * 50)
    
    return result

# Test with a sample question
sample_question = "What were the key trends in the stock market during 2024?"
test_result = test_agent_query(sample_question)


🔍 Testing Query: 'What were the key trends in the stock market during 2024?'
🧠 Calling LLM for reasoning...
💬 LLM response type: AIMessage
🔧 LLM wants to use 1 tools
🤔 Should continue? True
🔧 Executing tool: retriever_tool
📝 Query: key trends in stock market 2024
✅ Tool executed successfully. Result length: 4664 characters
🔄 Returning 1 tool results to LLM
🧠 Calling LLM for reasoning...
💬 LLM response type: AIMessage
🤔 Should continue? False

🤖 Agent Response:
The key trends in the stock market during 2024 were characterized by a strong performance driven predominantly by the technology sector. Here are the main highlights:

1. **Tech-Dominated Rally**: 2024 was marked by a tech-dominated rally, with significant wealth gains concentrated in the technology sector. This was driven by the adoption of next-generation technologies such as artificial intelligence, cloud computing, and quantum technology. The market saw substantial performances from mega-cap tech companies like Apple, Alphab

**🧪 What to expect:**
- The agent will search the document for relevant information
- It will provide a comprehensive answer with citations
- You'll see the tool execution logs in real-time

## 12. Interactive Agent Interface 

Create an interactive interface to chat with your agent:

In [29]:
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    
    def create_interactive_widget():
        """
        Creates an interactive widget interface for the RAG agent
        Requires: pip install ipywidgets
        """
        # Create widgets
        question_input = widgets.Textarea(
            placeholder="Enter your question about stock market performance...",
            description="Question:",
            layout=widgets.Layout(width='70%', height='80px')
        )
        
        ask_button = widgets.Button(
            description="Ask Agent",
            button_style='primary',
            icon='search'
        )
        
        output_area = widgets.Output()
        
        def on_ask_button_click(b):
            with output_area:
                clear_output(wait=True)
                if question_input.value.strip():
                    print(f"🔍 Question: {question_input.value}")
                    print("=" * 50)
                    print("🔄 Processing...")
                    
                    try:
                        messages = [HumanMessage(content=question_input.value)]
                        result = rag_agent.invoke({"messages": messages})
                        
                        print("\n🤖 AGENT RESPONSE:")
                        print("-" * 30)
                        print(result['messages'][-1].content)
                        print("=" * 50)
                        
                    except Exception as e:
                        print(f"❌ Error: {e}")
                else:
                    print("⚠️ Please enter a question!")
        
        ask_button.on_click(on_ask_button_click)
        
        # Display the interface
        display(widgets.VBox([
            widgets.HTML("<h3>🤖 RAG Agent - Stock Market Assistant</h3>"),
            question_input,
            ask_button,
            output_area
        ]))
    
    # Uncomment to create the widget interface:
    create_interactive_widget()
    
except ImportError:
    print("ipywidgets not available. Use the simple ask_agent() function instead.")
    
# Example question: What were the key stock market trends in 2024? 



**💡 Usage Tips:**

- Ask specific questions about stock market data
- Request comparisons between different time periods
- Ask for trends, analysis, or specific metrics
- The agent will search and cite relevant document sections



## 13. Advanced Features
**Multiple Document Support**
To extend this agent for multiple documents:

In [32]:
def load_multiple_documents(pdf_paths: list):
    """
    Example function to load multiple PDF documents
    """
    all_pages = []
    
    for pdf_path in pdf_paths:
        if os.path.exists(pdf_path):
            loader = PyPDFLoader(pdf_path)
            pages = loader.load()
            all_pages.extend(pages)
            print(f"✅ Loaded: {pdf_path} ({len(pages)} pages)")
        else:
            print(f"⚠️ File not found: {pdf_path}")
    
    return all_pages

# Example usage (uncomment and modify paths as needed):
document_paths = [
    "Stock_Market_Performance_2024.pdf",
    "U.S._Economic_Outlook.pdf"
]
all_documents = load_multiple_documents(document_paths)

✅ Loaded: Stock_Market_Performance_2024.pdf (9 pages)
✅ Loaded: U.S._Economic_Outlook.pdf (29 pages)


**Enhanced Search Strategies**

In [33]:
def create_advanced_retriever(vectorstore, search_type="mmr"):
    """
    Create retriever with Maximum Marginal Relevance for diverse results
    """
    return vectorstore.as_retriever(
        search_type=search_type,  # "mmr" for diverse results
        search_kwargs={
            "k": 6,              # Return more results
            "fetch_k": 20,       # Consider more candidates
            "lambda_mult": 0.7   # Balance relevance vs diversity
        }
    )

# Example: Enhanced retriever (uncomment to use)
enhanced_retriever = create_advanced_retriever(vectorstore)

## 14. Key Concepts Summary
**🎯 What We Built**

- Document Processing Pipeline: PDF → Text Chunks → Embeddings → Vector Store
- Retrieval System: Semantic search for relevant information
- Agent Architecture: LLM + Tools + State Management
- Interactive Interface: User-friendly query system

**🔑 Key Components**
| Component | Purpose | Technology |
|-----------|---------|------------|
| **PDF Loader** | Extract text from documents | PyPDFLoader |
| **Text Splitter** | Create manageable chunks | RecursiveCharacterTextSplitter |
| **Embeddings** | Convert text to vectors | OpenAI text-embedding-3-small |
| **Vector Store** | Store and search embeddings | ChromaDB |
| **LLM** | Reasoning and response generation | GPT-4o |
| **Agent Framework** | Orchestrate workflow | LangGraph |

**🚀 Capabilities Achieved**

- ✅ Accurate Information Retrieval: Finds relevant document sections
- ✅ Contextual Understanding: Maintains conversation context
- ✅ Multi-step Reasoning: Can make multiple searches if needed
- ✅ Source Citation: References specific document parts
- ✅ Interactive Interface: User-friendly question-answering

**🔮 Next Steps**

- Add More Tools: Web search, calculator, database queries, voice agent
- Improve Chunking: Experiment with different strategies
- Multi-modal Support: Add image and table processing
- Evaluation Metrics: Implement retrieval and response quality metrics
- Production Deployment: Add error handling, logging, and monitoring

**📚 Further Learning**

- LangChain Documentation: langchain.com
- LangGraph Tutorials: langgraph.com
- RAG Best Practices: Advanced chunking and retrieval strategies
- Agent Design Patterns: Multi-agent systems and tool composition


🎉 Congratulations! You've successfully built a complete RAG agent that can intelligently search through documents and provide informed responses. This foundation can be extended to handle multiple documents, different file types, and more sophisticated reasoning tasks.